# Training for pseudo labels
As the first part of the overall training, we train a DeepLabV2 model and get pseudo labels. In this notebook, the model will be trained with grayscale source and target images.
Cross entropy loss is used for training on grayscale source images. For grayscale target images, entropy minimization is performed. 
To load images in grayscale format, we will use datagray as dataloader instead of data as in the other notebooks.

In [1]:
import numpy as np
from utils.timer import Timer
import os
from datagray import CreateSrcDataLoader
from datagray import CreateTrgDataLoader
from model import CreateModel
import torch.backends.cudnn as cudnn
import torch
from torch.autograd import Variable
from utils import FDA_source_to_target
import scipy.io as sio
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [2]:
IMG_MEAN = np.array((104.00698793, 116.66876762, 122.67891434), dtype=np.float32)
IMG_MEAN = torch.reshape(torch.from_numpy(IMG_MEAN), (1, 3, 1, 1))

# weighting fence, light, sign, person, rider, bus, train, motorcycle, bicycle
CS_weights = np.array((1.0, 1.0, 1.0, 1.0, 100.0, 1.0, 100.0, 100.0, 1.0, 1.0, 1.0,
                       100.0, 100.0, 1.0, 1.0, 100.0, 500.0, 500.0, 1000.0), dtype=np.float32)
weight_normalizer = CS_weights.max()/2
CS_weights = torch.from_numpy(CS_weights/weight_normalizer)

## Train options
Parameters for the training process such as beta, ita, temperature as well as the directories to load/save model weights can be adjusted here. 

In [3]:
import argparse
import os.path as osp

class TrainOptions():
    def initialize(self):
        parser = argparse.ArgumentParser( description="training script for FDA" )
        parser.add_argument("--model", type=str, default='DeepLab', help="available options : DeepLab and VGG")
        parser.add_argument("--LB", type=float, default=0.1, help="beta for FDA")
        parser.add_argument("--GPU", type=str, default='0', help="which GPU to use")
        parser.add_argument("--entW", type=float, default=0.005, help="weight for entropy")
        parser.add_argument("--ita", type=float, default=2.0, help="ita for robust entropy")
        parser.add_argument("--temperature", type=float, default=0.07, help="temperature for contrastive loss")
        parser.add_argument("--switch2entropy", type=int, default=0, help="switch to entropy after this many steps")
        parser.add_argument("--switch2contrast", type=int, default=0, help="switch to contrastive learning  after this many steps")
        parser.add_argument('--threshold', default=0.95, type=float, help='pseudo label threshold')
        parser.add_argument("--source", type=str, default='gta5', help="source dataset : gta5 or synthia")
        parser.add_argument("--target", type=str, default='cityscapes', help="target dataset : cityscapes")
        parser.add_argument("--snapshot-dir", type=str, default='../checkpoints/UDA/model1', help="Where to save snapshots of the model.")
        parser.add_argument("--data-dir", type=str, default='../data/GTA5', help="Path to the directory containing the source dataset.")
        parser.add_argument("--data-list", type=str, default='./dataset/gta5_list/train_all.txt', help="Path to the listing of images in the source dataset.")
        parser.add_argument("--data-dir-target", type=str, default='../data/cityscapes', help="Path to the directory containing the target dataset.")
        parser.add_argument("--data-list-target", type=str, default='./dataset/cityscapes_list/train.txt', help="list of images in the target dataset.")
        parser.add_argument("--set", type=str, default='train', help="choose adaptation set.")
        parser.add_argument("--label-folder", type=str, default=None, help="Path to the directory containing the pseudo labels.")

        parser.add_argument("--batch-size", type=int, default=1, help="input batch size.")
        parser.add_argument("--num-steps", type=int, default=100000, help="Number of training steps.")
        parser.add_argument("--num-steps-stop", type=int, default=100000, help="Number of training steps for early stopping.")
        parser.add_argument("--num-workers", type=int, default=4, help="number of threads.")
        parser.add_argument("--learning-rate", type=float, default=2.5e-4, help="initial learning rate for the segmentation network.")
        parser.add_argument("--momentum", type=float, default=0.9, help="Momentum component of the optimiser.")
        parser.add_argument("--weight-decay", type=float, default=0.0005, help="Regularisation parameter for L2-loss.")
        parser.add_argument("--power", type=float, default=0.9, help="Decay parameter to compute the learning rate (only for deeplab).")

        parser.add_argument("--num-classes", type=int, default=19, help="Number of classes for cityscapes.")
        parser.add_argument("--init-weights", type=str, default='../checkpoints/DeepLab_init.pth', help="initial model.")
        parser.add_argument("--restore-from", type=str, default=None, help="Where restore model parameters from.")
        parser.add_argument("--save-pred-every", type=int, default=1000, help="Save summaries and checkpoint every often.")
        parser.add_argument("--print-freq", type=int, default=100, help="print loss and time fequency.")
        parser.add_argument("--matname", type=str, default='loss_log.mat', help="mat name to save loss")
        parser.add_argument("--tempdata", type=str, default='tempdata.mat', help="mat name to save data")


        return parser.parse_args(args=[])
    
    def print_options(self, args):
        message = ''
        message += '----------------- Options ---------------\n'
        for k, v in sorted(vars(args).items()):
            comment = ''
            message += '{:>25}: {:<30}{}\n'.format(str(k), str(v), comment)
        message += '----------------- End -------------------'
        print(message)
    
        # save to the disk
        file_name = osp.join(args.snapshot_dir, 'opt.txt')
        with open(file_name, 'wt') as args_file:
            args_file.write(message)
            args_file.write('\n')

In [4]:
opt = TrainOptions()
args = opt.initialize()
os.environ["CUDA_VISIBLE_DEVICES"] = args.GPU
_t = {'iter time': Timer()}

model_name = args.source + '_to_' + args.target
if not os.path.exists(args.snapshot_dir):
    os.makedirs(args.snapshot_dir)
    os.makedirs(os.path.join(args.snapshot_dir, 'logs'))
opt.print_options(args)

sourceloader, targetloader = CreateSrcDataLoader(args), CreateTrgDataLoader(args)
sourceloader_iter, targetloader_iter = iter(sourceloader), iter(targetloader)
print("model is created")
model, optimizer = CreateModel(args)

----------------- Options ---------------
                      GPU: 0                             
                       LB: 0.1                           
               batch_size: 1                             
                 data_dir: ../data/GTA5                  
          data_dir_target: ../data/cityscapes            
                data_list: ./dataset/gta5_list/train_all.txt
         data_list_target: ./dataset/cityscapes_list/train.txt
                     entW: 0.005                         
             init_weights: ../checkpoints/DeepLab_init.pth
                      ita: 2.0                           
             label_folder: None                          
            learning_rate: 0.00025                       
                  matname: loss_log.mat                  
                    model: DeepLab                       
                 momentum: 0.9                           
              num_classes: 19                            
                num_s

/home/singh/anaconda3/envs/adl4cv/lib/python3.8/site-packages/torch/optim/sgd.py:69: UserWarning: optimizer contains a parameter group with duplicate parameters; in future, this will cause an error; see github.com/pytorch/pytorch/issues/40967 for more information
  super(SGD, self).__init__(params, defaults)


In [5]:
start_iter = 0

# if the model is restored, get the last iterations as start_iter
if args.restore_from is not None:
    start_iter = int(args.restore_from.rsplit('/', 1)[1].rsplit('_')[1])

cudnn.enabled = True
cudnn.benchmark = True

model.train()
model.cuda()

# losses to log
loss_all = 0
loss_train = 0
loss_val = 0.0
loss_ent = 0

best_loss_trg = float('inf')
loss_val_list = []

mean_img = torch.zeros(1, 1)
class_weights = Variable(CS_weights).cuda()
_t['iter time'].tic()

## Training the model

In [ ]:
for i in range(start_iter, 30000):
    model.adjust_learning_rate(args, optimizer, i)  # adjust learning rate
    optimizer.zero_grad()  # zero grad
    
    src_img, src_lbl, _, _ = sourceloader_iter.next()  # new batch source
    trg_img, trg_lbl, _, _ = targetloader_iter.next()  # new batch target
    
    if mean_img.shape[-1] < 2:
        B, C, H, W = trg_img.shape
        mean_img = IMG_MEAN.repeat(B, 1, H, W)


    
    # 1. forward pass source image 
    src_img = src_img - mean_img
    src_img, src_lbl = Variable(src_img).cuda(), Variable(src_lbl.long()).cuda()  # to gpu
    src_seg_score = model(src_img, lbl=src_lbl, weight=class_weights, ita=args.ita)  # forward pass
    loss_seg_src = model.loss_seg  # get segmentation loss

    # 2. forward pass target image 
    trg_img, trg_lbl = Variable(trg_img).cuda(), Variable(trg_lbl.long()).cuda()  # to gpu
    trg_seg_score = model(trg_img, lbl=trg_lbl, weight=class_weights, ita=args.ita)  # forward pass
    loss_seg_trg = model.loss_seg  # get segmentation loss
    loss_ent_trg = model.loss_ent # get entropy 
    
    # sum segmentation loss on source data and entropy on target data to backpropagate 
    loss_all = loss_seg_src + args.entW*loss_ent_trg
    loss_all.backward()
    optimizer.step()

    
    loss_train += loss_seg_src.detach().cpu().numpy()
    loss_val += loss_seg_trg.detach().cpu().numpy()
    loss_ent += loss_ent_trg.detach().cpu().numpy()
    
    # save the model weights for each save_pred_every steps
    if (i + 1) % args.save_pred_every == 0:
        print('taking snapshot ...')
        torch.save(model.state_dict(), os.path.join(args.snapshot_dir, '%s_' % (args.source) + str(i + 1) + '.pth'))
        
    # save the model with smallest validation loss as the best model
    if best_loss_trg > loss_seg_trg:
        best_loss_trg = loss_seg_trg
        torch.save(model.state_dict(), os.path.join(args.snapshot_dir, '%s_' % (args.source) +'best'+ '.pth'))
    
    if (i + 1) % args.print_freq == 0:
        _t['iter time'].toc(average=False)
        print('[it %d][src seg loss %.4f][trg seg loss %.4f][lr %.4f][%.2fs]' % \
              (i + 1,loss_seg_src.data, loss_seg_trg.data, optimizer.param_groups[0]['lr'] * 10000,
               _t['iter time'].diff))
    
        sio.savemat(args.tempdata, {'trg_img': trg_img.cpu().numpy()})
        
        loss_train /= args.print_freq
        loss_val /= args.print_freq
        loss_val_list.append(loss_val)
        loss_ent /= args.print_freq
        
        writer.add_scalar('Training loss', loss_train, i)
        writer.add_scalar('Validation loss', loss_val, i )
        writer.add_scalar('Entropy loss', loss_ent, i)
        
        sio.savemat(args.matname, {'loss_val': loss_val_list})
        loss_train = 0.0
        loss_val = 0.0
        loss_ent = 0.0
        
        if i + 1 > args.num_steps_stop:
            print('finish training')
            break
        _t['iter time'].tic()

/home/singh/anaconda3/envs/adl4cv/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


[it 100][src seg loss 2.8874][trg seg loss 4.5537][lr 2.4978][109.82s]
[it 200][src seg loss 1.5248][trg seg loss 3.2787][lr 2.4955][105.94s]
[it 300][src seg loss 1.8459][trg seg loss 2.1203][lr 2.4933][107.19s]
[it 400][src seg loss 1.3691][trg seg loss 4.2199][lr 2.4910][106.26s]
[it 500][src seg loss 1.3471][trg seg loss 4.4983][lr 2.4888][105.20s]
[it 600][src seg loss 0.8436][trg seg loss 2.0812][lr 2.4865][104.98s]
[it 700][src seg loss 0.7600][trg seg loss 5.2852][lr 2.4843][105.01s]
[it 800][src seg loss 1.9242][trg seg loss 5.1813][lr 2.4820][106.95s]
[it 900][src seg loss 1.0364][trg seg loss 1.3531][lr 2.4798][105.37s]
taking snapshot ...
[it 1000][src seg loss 1.0378][trg seg loss 2.0518][lr 2.4775][106.02s]
